In [1]:
import numpy as np
import random
import scipy.io as sio
import os

In [2]:
import sys
sys.path.append("D:/CurrentlyActiveResearch/oilspill_project/utils/") # path file zeroPadding

from zeroPadding import zeroPadding_3D

In [3]:
class HSIData:
    def __init__(self, img, gt):
        self.img = img      # hyperspectral cube (H, W, B)
        self.gt  = gt       # ground truth (H, W)

In [4]:
def createSample(hsi, patch_size, num_per_class):
    """
    Ambil sampel patch dari data hyperspectral.
    - hsi.img : data hyperspectral (H, W, B)
    - hsi.gt  : ground truth (H, W) berisi label piksel
    - patch_size : ukuran patch (patch 9x9)
    - num_per_class : jumlah patch yang diambil per kelas
    """
    training_hsi = hsi
    training_hsi_gt = training_hsi.gt
    print("hsi shape:", training_hsi.img.shape)

    # Mencari semua koordinat piksel kelas 0 dan kelas 1
    indices_0 = np.where(training_hsi_gt == 0)
    indices_1 = np.where(training_hsi_gt == 1)

    zero_indices = list(zip(indices_0[0], indices_0[1]))
    one_indices  = list(zip(indices_1[0], indices_1[1]))

    # Mengambil koordinat random sebanyak num_per_class untuk setiap kelas (0 dan 1)
    random_indices_0 = random.sample(zero_indices, k=num_per_class)
    random_indices_1 = random.sample(one_indices,  k=num_per_class)

    print(f"Creating {num_per_class} samples for class 0 and class 1...")

    # Menyiapkan wadah patch berupa array 2 dimensi berisi 0
    data = training_hsi.img
    n_bands = data.shape[2]
    half_patch = patch_size // 2

    selected_patch_0 = np.zeros((num_per_class, patch_size, patch_size, n_bands))
    selected_patch_1 = np.zeros((num_per_class, patch_size, patch_size, n_bands))

    # Menambahkan padding supaya patch bisa diambil di tepi gambar
    matrix = zeroPadding_3D(data, half_patch)

    # Mengambil patch kelas 0
    for i, (x_pos, y_pos) in enumerate(random_indices_0):
        patch = matrix[
            x_pos : x_pos + 2*half_patch + 1,
            y_pos : y_pos + 2*half_patch + 1,
            :
        ]
        selected_patch_0[i] = patch

    # Mengambil patch kelas 1
    for i, (x_pos, y_pos) in enumerate(random_indices_1):
        patch = matrix[
            x_pos : x_pos + 2*half_patch + 1,
            y_pos : y_pos + 2*half_patch + 1,
            :
        ]
        selected_patch_1[i] = patch

    return selected_patch_0, selected_patch_1, random_indices_0, random_indices_1


In [5]:
mat = sio.loadmat("D:/CurrentlyActiveResearch/oilspill_project/data/raw/GM01.mat") # path data
img = mat["img"]   # (H, W, B)
gt  = mat["map"]    # (H, W)

hsi = HSIData(img, gt)
print("Image shape:", img.shape, "GT shape:", gt.shape)

Image shape: (1243, 684, 224) GT shape: (1243, 684)


In [6]:
patch_size = 9
num_per_class = 5

selected_patch_0, selected_patch_1, idx0, idx1 = createSample(
    hsi, patch_size, num_per_class
)

print("Class 0 patches:", selected_patch_0.shape)
print("Class 1 patches:", selected_patch_1.shape)

hsi shape: (1243, 684, 224)
Creating 5 samples for class 0 and class 1...
Class 0 patches: (5, 9, 9, 224)
Class 1 patches: (5, 9, 9, 224)


In [ ]:
save_dir = "D:/CurrentlyActiveResearch/oilspill_project/data/processed/"
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, "patch_class0.npy"), selected_patch_0)
np.save(os.path.join(save_dir, "patch_class1.npy"), selected_patch_1)

print("Patches saved in:", save_dir)